In [ ]:
import requests
from pydub import AudioSegment

# VoicevoxエンジンのURL
BASE_URL = "http://127.0.0.1:50021"

def synthesize_voice(text, speaker, output_file):
    """
    Voicevoxを使って音声を生成し、指定したファイルに保存
    :param text: 合成するテキスト
    :param speaker: スピーカーID
    :param output_file: 保存先ファイルパス
    """
    # 音声クエリを生成
    response = requests.post(f"{BASE_URL}/audio_query", params={"text": text, "speaker": speaker})
    if response.status_code != 200:
        print("クエリ生成に失敗:", response.text)
        return None

    query = response.json()

    # 音声を生成
    response = requests.post(f"{BASE_URL}/synthesis", json=query, params={"speaker": speaker})
    if response.status_code != 200:
        print("音声生成に失敗:", response.text)
        return None

    # ファイルに保存
    with open(output_file, "wb") as f:
        f.write(response.content)

    return output_file

def generate_conversation(conversation_script, output_file="conversation.wav"):
    """
    会話のスクリプトを基に音声を生成し、1つの音声ファイルにまとめる
    :param conversation_script: 会話スクリプト (リスト形式)
    :param output_file: 保存する最終的な音声ファイル
    """
    # 各セリフの音声を生成
    audio_segments = []
    for i, line in enumerate(conversation_script):
        text = line["text"]
        speaker = line["speaker"]
        temp_file = f"../wav/test_{i}.wav"
        synthesize_voice(text, speaker, temp_file)
        segment = AudioSegment.from_file(temp_file)
        audio_segments.append(segment)

    # 全ての音声を結合
    # combined_audio = sum(audio_segments)
    silence = AudioSegment.silent(duration=500)  # 0.5秒の無音
    combined_audio = sum([segment + silence for segment in audio_segments])


    # ファイルに保存
    combined_audio.export(output_file, format="wav")
    print(f"会話音声を保存しました: {output_file}")

# 実行例
if __name__ == "__main__":
    conversation_script = [
        {"text": "こんにちは、今日は何をしていますか？", "speaker": 1},  # 四国めたん
        {"text": "こんにちは！少し散歩をしていました。", "speaker": 2},  # ずんだもん
        {"text": "いいですね！天気も良いですし、気持ちよさそうです。", "speaker": 1},
        {"text": "はい、とても気分転換になりました！", "speaker": 2},
    ]
    generate_conversation(conversation_script, "../wav/conversation.wav")


会話音声を保存しました: conversation.wav
